In [1]:
import sys
import pandas as pd
import os
from keplergl import KeplerGl
from tqdm import tqdm

import pickle
from h3 import h3

def load_pickle(file_path):
    with open(file_path, 'rb') as handle:
        return pickle.load(handle)
        
def save_pickle(obj, filepath): 
    with open(filepath, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
os.makedirs('configs', exist_ok=True)

In [3]:
df_roads = pd.read_csv("/Users/dashavecherinka/Downloads/train/roads_dataset.csv").drop_duplicates()

df_transport = pd.read_csv("/Users/dashavecherinka/Downloads/train/routes_dataset.csv").drop_duplicates()

df_population = pd.read_csv("/Users/dashavecherinka/Downloads/train/rosstat_population_all_cities.csv").drop_duplicates()
df_population = df_population.set_index(df_population['geo_h3_10'].values)

df_isochrones = pd.read_csv("/Users/dashavecherinka/Downloads/train/isochrones_walk_dataset.csv").drop_duplicates()
df_isochrones = df_isochrones.set_index(df_isochrones['geo_h3_10'].values)

df_stops = pd.read_csv("/Users/dashavecherinka/Downloads/train/osm_stops.csv").drop_duplicates()
df_stops = df_stops.set_index(df_stops['geo_h3_10'].values)

df_companies = pd.read_csv("/Users/dashavecherinka/Downloads/train/osm_amenity.csv").drop_duplicates()
df_companies = df_companies.set_index(df_companies['geo_h3_10'].values)

df_isochrones2 = pd.read_csv("/Users/dashavecherinka/Downloads/train/isochrones_drive_dataset.csv").drop_duplicates()
df_isochrones2 = df_isochrones2.set_index(df_isochrones2['geo_h3_10'].values)

df_ATM = pd.read_csv("/Users/dashavecherinka/Downloads/train/target_hakaton_spb.csv", sep = ';').drop_duplicates()
df_ATM = df_ATM.set_index(df_ATM['geo_h3_10'].values)

In [4]:
# Пересчет для роя
def location(category,city):
        #определяем город
    df_ATM_test = df_ATM[df_ATM['gorod'] == 'Санкт-Петербург']
    df_isochrones_test = df_isochrones[df_isochrones['city'] == 'Санкт-Петербург']
    df_ATM_test = df_ATM_test[df_ATM_test['atm_category'] == category]
    df_stops_test = df_stops[df_stops['city'] == 'Санкт-Петербург'] 
    df_companies_test= df_companies[df_companies['city'] == 'Санкт-Петербург']
    df_companies_test2 = df_companies[df_companies['city'] == 'Санкт-Петербург']
    df_companies_test2 = df_companies_test2.drop(columns='Банки')  
    index = df_ATM_test[df_ATM_test['atm_category'] == category ]['target'].sort_values()[-20:].keys()

    #определяем таргет
    index = df_ATM_test[df_ATM_test['atm_category'] == category]['target'].sort_values()[-20:].keys()
    # Какие хексагоны входят в район
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    h3_geo={}
    for i in index:
        if i in df_isochrones_test['geo_h3_10'][:]:
            h3_geo[i] = list()
            h3_geo[i].append(df_isochrones_test['geo_h3_10'][i])
            temp0 = []
            for j in range(len(df_isochrones_test['walk_15min'][i][10:][:-2].split(","))):
                temp0.append(tuple(float(x) for x in df_isochrones_test['walk_15min'][i][10:][:-2].split(",")[j].split())) 
            polygon = Polygon(temp0)
            for j in df_isochrones_test['geo_h3_10'][:]:
                point = Point(df_isochrones_test['lon'][j], df_isochrones_test['lat'][j])
                if polygon.contains(point) == True:
                    h3_geo[i].append(df_isochrones_test['geo_h3_10'][j])
    from tqdm import tqdm
    import numpy as np
    # Численность по районам
    h3={}
    population_list=[]
    atm_cnt=[]
    stop_list=[]
    comp_list=[]
    bank_list=[]
    for i in h3_geo.keys():
        h3[i]=list()
        sum_pop = 0
        sum_count=0
        sum_stop=0
        sum_banks=0
        for j in list(h3_geo[i]):
            if j in df_population['geo_h3_10'][:]:
                sum_pop+=df_population['population'][j]
            if j in df_ATM_test['geo_h3_10'][:]:
                sum_count+=np.nan_to_num(df_ATM_test['atm_cnt'][j])
            if j in df_stops_test['geo_h3_10'][:]:
                sum_stop+=1
            if j in df_companies_test['geo_h3_10'][:]:
                sum_banks+=np.nan_to_num(df_companies_test['Банки'][j])

        h3[i].append(sum_pop)
        h3[i].append(sum_count)
        h3[i].append(sum_banks)
        h3[i].append(sum_stop)

        population_list.append(sum_pop)
        atm_cnt.append(sum_count)
        stop_list.append(sum_stop)
        bank_list.append(sum_banks)

    max1=max(population_list)
    max2=max(atm_cnt)
    max3=max(stop_list)
    max4=max(bank_list)


    population_list = population_list/max1
    atm_cnt = atm_cnt/max2
    stop_list = np.array(stop_list)/(max3+0.00000000001)
    bank_list = bank_list/(max4+0.00000000001)


    import statistics 
    etalon_pop = np.mean(population_list)
    etalon_atm_cnt = statistics.mode(atm_cnt)
    etalon_bank_list = np.mean(bank_list)
    etalon_stop_list = np.mean(stop_list)
    #параметры
    particles_amout = 50
    iterations_amount = 60
    n = 3
    mu = 3
    lmbd = 2
    w1 = 0.1
    crossoverProbability = 0.5 
    mutationProbability = 0.1
    counter=0

    #определяем город
    df_ATM_test = df_ATM[df_ATM['gorod'] == city]
    df_isochrones_test = df_isochrones[df_isochrones['city'] == city]
    df_ATM_test = df_ATM_test[df_ATM_test['atm_category'] == category]
    df_stops_test = df_stops[df_stops['city'] == city]
    df_companies_test=df_companies[df_companies['city'] == city]
    #выбор случайных значений роя
    import random
    particles = random.choices(df_isochrones_test['geo_h3_10'], k = particles_amout)
    particles_position = {}
    for i in particles :
        particles_position[i] = list()
        particles_position[i].append(df_isochrones_test['lat'][i])
        particles_position[i].append(df_isochrones_test['lon'][i])

    #расчет значений целевой функции

    best_particles_value_frame = pd.DataFrame.from_dict(particles_position).T
    best_particles_value_frame = best_particles_value_frame.rename(columns = {0:'lat',1:'lon',2:'func'})

    # Какие хексагоны входят в район
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    h3_geo={}
    for i in particles:
        if i in df_isochrones_test['geo_h3_10'][:]:
            h3_geo[i] = list()
            h3_geo[i].append(df_isochrones_test['geo_h3_10'][i])
            temp0 = []
            for j in range(len(df_isochrones_test['walk_15min'][i][10:][:-2].split(","))):
                temp0.append(tuple(float(x) for x in df_isochrones_test['walk_15min'][i][10:][:-2].split(",")[j].split())) 
            polygon = Polygon(temp0)
            for j in df_isochrones_test['geo_h3_10'][:]:
                point = Point(df_isochrones_test['lon'][j], df_isochrones_test['lat'][j])
                if polygon.contains(point) == True:
                    h3_geo[i].append(df_isochrones_test['geo_h3_10'][j])
    # Численность по районам
    h3={}
    population_list=[]
    atm_cnt=[]
    stop_list=[]
    comp_list=[]
    bank_list=[]
    for i in h3_geo.keys():
        h3[i]=list()
        sum_pop = 0
        sum_count=0
        sum_stop=0
        sum_banks=0
        for j in list(h3_geo[i]):
            if j in df_population['geo_h3_10'][:]:
                sum_pop+=df_population['population'][j]
            if j in df_ATM_test['geo_h3_10'][:]:
                sum_count+=np.nan_to_num(df_ATM_test['atm_cnt'][j])
            if j in df_stops_test['geo_h3_10'][:]:
                sum_stop+=1
            if j in df_companies_test['geo_h3_10'][:]:
                sum_banks+=np.nan_to_num(df_companies_test['Банки'][j])

        h3[i].append(sum_pop)
        h3[i].append(sum_count)
        h3[i].append(sum_banks)
        h3[i].append(sum_stop)

        population_list.append(sum_pop)
        atm_cnt.append(sum_count)
        stop_list.append(sum_stop)
        bank_list.append(sum_banks)

    max1=max(population_list)
    max2=max(atm_cnt)
    max3=max(stop_list)
    max4=max(bank_list)


    population_list = population_list/max1
    atm_cnt = atm_cnt/max2
    stop_list = np.array(stop_list)/(max3+0.00000000001)
    bank_list = np.array(bank_list)/(max4+0.00000000001)

    #Пересчет значения целевых функций
    func = 1.6 * abs(population_list - etalon_pop) + 1 * abs(atm_cnt - etalon_atm_cnt) + \
    1.2 * abs(stop_list - etalon_stop_list) + 1.4 * abs(bank_list - etalon_bank_list)
    j = 0
    for i in particles:
        particles_position[i].append(func[j])
        j+=1

    while counter < iterations_amount:

        best_particles_value_frame = pd.DataFrame.from_dict(particles_position).T
        best_particles_value_frame = best_particles_value_frame.rename(columns = {0:'lat',1:'lon',2:'func'})
        # выбор родителей
        parents_list = []
        best_rand_parent =[(best_particles_value_frame['func'][i],i)\
         for i in random.choices(best_particles_value_frame['func'].keys(), k = 2*(mu-1))]
        best_rand_parent.sort(key=lambda x: x[0])
        best_rand_parent1 = best_rand_parent[0][1]
        best_rand_parent2 = best_rand_parent[1][1]
        best_rand_parent3 = best_particles_value_frame['func'].sort_values().keys()[0]
        parents_list.append(best_rand_parent1)
        parents_list.append(best_rand_parent2)
        parents_list.append(best_rand_parent3)

        crossover_operator = [[0,0],[0,0]]

        while crossover_operator[0] == crossover_operator[1]:
            # кроссовер
            crossover_operator = []
            for i in range(lmbd):
                crossover_operator0 = []
                crossoverNumber = random.random()
                if crossoverNumber > (1 - crossoverProbability):
                    glat = np.mean([best_particles_value_frame['lat'][parents_list[0]],\
                        best_particles_value_frame['lat'][parents_list[1]],\
                        best_particles_value_frame['lat'][parents_list[2]]])
                    dlat =  best_particles_value_frame['lat'][parents_list[2]] - glat 
                    if dlat != 0:
                        w2 = abs(dlat)
                    else: 
                        w2 = 0.1
                    crossover_operator0.append(best_particles_value_frame['lat'][parents_list[0]] + \
                        w1 * dlat + w2 * ((best_particles_value_frame['lat'][parents_list[1]] - best_particles_value_frame['lat'][parents_list[2]])/2))
                    glon = np.mean([best_particles_value_frame['lon'][parents_list[0]],\
                        best_particles_value_frame['lon'][parents_list[1]],\
                        best_particles_value_frame['lon'][parents_list[2]]])
                    dlon =  best_particles_value_frame['lon'][parents_list[2]] - glon 
                    if dlon != 0:
                        w2 = abs(dlon)
                    else: 
                        w2 = 0.1
                    crossover_operator0.append(best_particles_value_frame['lon'][parents_list[0]] + \
                            w1 * dlat + w2 * ((best_particles_value_frame['lon'][parents_list[1]] - best_particles_value_frame['lon'][parents_list[2]])/2))
                else:
                    crossover_operator0 = list(random.choice(
                        ((best_particles_value_frame['lat'][parents_list[0]], best_particles_value_frame['lon'][parents_list[0]]),\
                         (best_particles_value_frame['lat'][parents_list[1]], best_particles_value_frame['lon'][parents_list[1]]),\
                        (best_particles_value_frame['lat'][parents_list[2]], best_particles_value_frame['lon'][parents_list[2]]))))
                crossover_operator.append(crossover_operator0)
            # полиномиальная мутация
            for i in range(lmbd):
                polynomial_mutation_number = random.random()
                if polynomial_mutation_number > (1 - mutationProbability):
                    nu = random.randrange(1,100)
                    r1 = random.random()
                    if r1 < 0.5:
                        delt = (2*r1)**((1/nu)+1) 
                    else:
                        delt = 1 - (2*(1-r1))**((1/nu)+1)
                    crossover_operator[i][0] = crossover_operator[i][0] + (best_particles_value_frame['lat'][parents_list[2]]- best_particles_value_frame['lat'][parents_list[1]])* delt
                    crossover_operator[i][1] = crossover_operator[i][1] + (best_particles_value_frame['lon'][parents_list[2]]- best_particles_value_frame['lon'][parents_list[1]])* delt
            if crossover_operator[0] == crossover_operator[1]:
                crossover_operator[1][0] = crossover_operator[1][0] + random.random()
                crossover_operator[1][1] = crossover_operator[1][1] + random.random()
                                
        import math
        def lat_lon(list):
            lat1 = list[0]
            lon1 = list[1]
            list3=[(math.sqrt((df_isochrones_test['lat'][i]-lat1)**2+(df_isochrones_test['lon'][i]-lon1)**2),i)\
                   for i in df_isochrones_test['geo_h3_10'][:] ]
            list3.sort(key=lambda x: x[0])
            return(list3[0][1],list3[1][1])
        offspring_list = []
        for i in range(len(crossover_operator)):
            lat_lon1 = lat_lon(crossover_operator[i])
            offspring_list.append(lat_lon1[0])
        if offspring_list[0]==offspring_list[1]:
            offspring_list[1]=lat_lon1[1]

        parent_for_ex = []
        rand_par = random.choices(list(particles_position.keys()), k = 1)[0]
        parent_for_ex.append(rand_par)
        offspring_list.append(rand_par)
        rand_par = random.choices(list(particles_position.keys()), k = 1)[0]
        parent_for_ex.append(rand_par)
        while parent_for_ex[0]==parent_for_ex[1]:
            parent_for_ex[1]=random.choices(list(particles_position.keys()), k = 1)[0]
        offspring_list.append(parent_for_ex[1])  

        from shapely.geometry import Point
        from shapely.geometry.polygon import Polygon
        h3_geo={}
        for i in offspring_list[:2]:
            if i in df_isochrones_test['geo_h3_10'][:]:
                h3_geo[i] = list()
                h3_geo[i].append(df_isochrones_test['geo_h3_10'][i])
                temp0 = []
                for j in range(len(df_isochrones_test['walk_15min'][i][10:][:-2].split(","))):
                    temp0.append(tuple(float(x) for x in df_isochrones_test['walk_15min'][i][10:][:-2].split(",")[j].split())) 
                polygon = Polygon(temp0)
                for j in df_isochrones_test['geo_h3_10'][:]:
                    point = Point(df_isochrones_test['lon'][j], df_isochrones_test['lat'][j])
                    if polygon.contains(point) == True:
                        h3_geo[i].append(df_isochrones_test['geo_h3_10'][j])
        # Численность по районам
        h3={}
        population_list=[]
        atm_cnt=[]
        stop_list=[]
        comp_list=[]
        bank_list=[]
        for i in h3_geo.keys():
            h3[i]=list()
            sum_pop = 0
            sum_count=0
            sum_stop=0
            sum_banks=0
            for j in list(h3_geo[i]):
                if j in df_population['geo_h3_10'][:]:
                    sum_pop+=df_population['population'][j]
                if j in df_ATM_test['geo_h3_10'][:]:
                    sum_count+=df_ATM_test['atm_cnt'][j]
                if j in df_stops_test['geo_h3_10'][:]:
                    sum_stop+=1
                if j in df_companies_test['geo_h3_10'][:]:
                    sum_banks+=np.nan_to_num(df_companies_test['Банки'][j])

            h3[i].append(sum_pop)
            h3[i].append(sum_count)
            h3[i].append(sum_banks)
            h3[i].append(sum_stop)

            population_list.append(sum_pop) 
            atm_cnt.append(sum_count)
            stop_list.append(sum_stop)
            bank_list.append(sum_banks) 
        #Пересчет значения целевых функций
        func = abs(population_list - etalon_pop) + abs(atm_cnt - etalon_atm_cnt) + \
        abs(stop_list - etalon_stop_list) + abs(bank_list - etalon_bank_list)
        j = 0
        offspring_list_data = {}
        for i in offspring_list:
            offspring_list_data[i] = list()
            if j < 2:
                offspring_list_data[i].append(df_isochrones_test['lat'][offspring_list[j]])
                offspring_list_data[i].append(df_isochrones_test['lon'][offspring_list[j]])
                offspring_list_data[i].append(func[j])
                j+=1
            else:
                offspring_list_data[i].append(particles_position[offspring_list[j]][0])
                offspring_list_data[i].append(particles_position[offspring_list[j]][1])
                offspring_list_data[i].append(particles_position[offspring_list[j]][2])
                j+=1
        del particles_position[offspring_list[2]]
        del particles_position[offspring_list[3]]
        b =[(offspring_list_data[i][2],i) for i in offspring_list_data.keys()]
        b.sort(key=lambda x: x[0])
        for i in range(2):
            particles_position[b[i][1]]=offspring_list_data[b[i][1]]
        counter+=1
    q = [(particles_position[i][0],particles_position[i][1],particles_position[i][2],i) for i in particles_position.keys()]
    q.sort(key=lambda x: x[0])
    return(q[0][0],q[0][1])
    

In [5]:
category_id=['category1', 'category2','category3','category4']
coordinat=[]
for i in category_id:
    coordinat.append(location(i,'Новосибирск'))
for i in range(4):   
    print("Координаты банкомата " , category_id[i] , " - ",coordinat[i])
    

Координаты банкомата  category1  -  (54.84847843465986, 82.97390668520336)
Координаты банкомата  category2  -  (54.8171302314008, 83.09541006946871)
Координаты банкомата  category3  -  (54.86157327583376, 82.97621963139348)
Координаты банкомата  category4  -  (54.84112661283599, 83.10007774967546)
